In [ ]:
// boring imports
var Plot = require('plotly-notebook-js');
var {loadLabelledWine, plotClustersWithLabels} = require('./utils');

# Supervised Learning


![Supervised learning](images/slide_supervised.png)

### Techniques availble

 - **KNN - K Nearest Neighbors**
 - SVM - Scalar Vector Machines
 - Naive Bayes
 - Partial Least Squares [regression]



In [ ]:
var {features, dataset} = loadLabelledWine({verbose: true});

## K Nearest Neighbors


##### KNN in mljs

 - docs for the KNN module are [here](https://mljs.github.io/knn/)
 - docs for the various kernel options are [here](https://github.com/mljs/kernel)

Make KNN with inputs and labels.

In [ ]:
var KNN = require('ml-knn');

var inputs = dataset.map(d => [d[0], d[10]]);
var labels = dataset.map(d => d[13]);

var options = {
  k: 3
};

var knn = new KNN(inputs, labels, options);

Check the some slices from labels and predicted values by using `knn.predict(inputs)`.

### Make some predictions and show some results

Let's plot the original dataset including it's known labels and our predictions

In [ ]:
var predictions = inputs.map(input => knn.predict(input))

In [ ]:
$$html$$ = plotClustersWithLabels(inputs.map(d => d[0]), inputs.map(d => d[1]), labels, "Actual Labels");

In [ ]:
$$html$$ = plotClustersWithLabels(inputs.map(d => d[0]), inputs.map(d => d[1]), predictions, "Predicted Labels");

### Measure Accuracy

Use the same confusion matrix approach as earlier to compute accuracy and f1-scores

In [ ]:
var ConfusionMatrix = require('ml-confusion-matrix');

var actuals = labels;
var predicted = inputs.map(i => knn.predict(i))

var C = ConfusionMatrix.fromLabels(actuals, predicted)

var M = C.getMatrix();
var trace = { 
    x: [0,1,2],
    y: [0,1,2],
    z: M,
    type: 'heatmap',
    showscale: false,
    colorscale:[[0, '#3D9970'], [100, '#001f3f']]
};

console.log("Accuracy", C.getAccuracy())
console.log("F1 Class 1", C.getF1Score(1))
console.log("F1 Class 2", C.getF1Score(2))
console.log("F1 Class 3", C.getF1Score(3))

// everything below here is just plotting code, get it in a utility fn!
var annotations = [];
M.map((a,y) => {
    a.map((b,x) => {
        annotations.push(
            {
                x: x,
                y: y,
                text: M[y][x],
                font: {
                    family: 'Arial',
                    size: 12,
                    color: 'white'
                  },
                showarrow: false
            }
        )
    })
})

var layout = { 
    xaxis: { title: "predicted", side: 'top' },
    yaxis: { title: "actuals", nticks: 6, autosize: false, autorange: 'reversed' },
    annotations,
    width: 500, height: 500};

$$html$$ = Plot.createPlot([trace], layout).render();

#### Discussion

 - How does you accuracy compare with our unsupervised approaches?
 - Is this too good to be tru and should you be suspicous?
 - Can you think of how to get a better accuracy measurement?